Setup

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from datetime import date
import numpy as np
import os
import pandas as pd
import sys
ancestor = os.path.abspath(os.path.join(os.pardir, os.pardir))
if ancestor not in sys.path:
    sys.path.append(ancestor)
from Biblio_Reader import manager as mg
existing = pd.read_csv('../outputs/FCP_DATA_updated_2017.csv')
existing.dropna(axis=0, how='all', inplace=True)
withsets = existing.loc[~pd.isnull(existing["Sets"])].copy()


def cdf_weighted(data, column, value):
    """
    Given a DataFrame, a column name, and a value in that column,
    returns a single weighted CiteScore value.
    
    Parameters
    ----------
    data : DataFrame
        needs "Journal" column
       
    column : string
        column to filter
       
    value : any dtype
        value in `column` to get value for
        
    Returns
    -------
    cdfw : float
        ∑CiteScore/n
    """
    j = mg.get_journal_attrs()
    l = [str(journal_name).lower() for journal_name in data.loc[data[column] == value]["Journal"]]
    citescores = [float(j[k]["CiteScore"]) for k in l if k in j and len(j[k]["CiteScore"])]
    return(float("%.2f" %round(np.mean(citescores), 2)))

# Contributor:

In [3]:
cdf_weighted(existing, "Contributor", "Contributor")

5.45

# Non-contributor:

In [4]:
cdf_weighted(existing, "Contributor", "Not a Contributor")

4.96

# ABIDE data use:

In [5]:
cdf_weighted(withsets.loc[withsets.Sets.str.contains("ABIDE")], "Data Use", "Y")

4.82

# Non-ABIDE data use:

In [6]:
cdf_weighted(withsets.loc[~withsets.Sets.str.contains("ABIDE")], "Data Use", "Y")

4.6